# Projeto de Redes Neurais 

## Referências: 

1. [Estudo sobre Câncer de Cólon utilizando a mesma base com modelod e CNN](https://www.kaggle.com/aayushrajput/lung-colon-cancer)
2. [Modelo de SqueezeNet para reconhecimento de comida](https://www.kaggle.com/kmader/food-squeezenet) 
3. [SqueezeNet no Keras](https://codelabs.developers.google.com/codelabs/keras-flowers-squeezenet#6)
4. [Entendendo Redes Convolucionais (CNNs) ](https://medium.com/neuronio-br/entendendo-redes-convolucionais-cnns-d10359f21184#:~:text=H%C3%A1%20muitas%20fun%C3%A7%C3%B5es%2C%20como%20sigmoid,quando%20comparada%20a%20outras%20fun%C3%A7%C3%B5es)
5. [Uma introdução as redes neurais convolucionais utilizando o Keras](https://medium.com/data-hackers/uma-introdu%C3%A7%C3%A3o-as-redes-neurais-convolucionais-utilizando-o-keras-41ee8dcc033e)
6. [Tutorial prático do Keras](https://cv-tricks.com/tensorflow-tutorial/keras/)
7. [Métricas de avaliação de modelo](https://gabrielschade.github.io/2019/03/12/ml-classificacao-metricas.html)


## Erro na biblioteca keras_applications

In [ ]:
!pip install keras_applications

## Importanto as bibliotecas necessárias

In [ ]:
import h5py
import numpy as np

import tensorflow as tf

from tensorflow.keras import datasets, layers, models

from keras import backend as K
from keras_applications.imagenet_utils import _obtain_input_shape
from keras.applications.vgg16 import VGG16R
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.engine.topology import get_source_inputs
from keras.models import Model
from keras.layers import Dense,Flatten
from keras.layers import Input, Activation, concatenate
from keras.layers import Flatten, Dropout
from keras.layers import Dense,Flatten
from keras.layers import Convolution2D, MaxPooling2D,Conv2D
from keras.layers import AveragePooling2D, BatchNormalization
from keras.layers import GlobalAveragePooling2D, GlobalMaxPooling2D
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import get_file
from keras.utils import layer_utils

import matplotlib.pyplot as plt

# from visual_callbacks import AccLossPlotter

from google.colab import drive
drive.mount('/content/drive', force_remount=True)
%matplotlib inline

Mounted at /content/drive


## Pré-processamento

### Variáveiras

In [ ]:
data_dir = '/content/drive/MyDrive/lung_image_sets/' 
width, height, nb_class = 768, 768, 3
batch_size = 4 
shuffle = True 
seed = 42 
nb_epoch = 1
loss='categorical_crossentropy'
optimizer_cnn='adam'
optimizer_sn = 'sgd'
metrics='accuracy'
target_size = (width, height)
input_shape =  (width, height, nb_class)

### Função de Pré-processamento

O conjunto de dados contém 15.000 imagens coloridas em 3 classes, com 5.000 imagens em cada classe. O conjunto de dados é dividido em 12.000 imagens de treinamento e 3.000 imagens de teste. 

In [ ]:
def PreProcess(data_dir,target_size,batch_size,shuffle,seed):
    
    datagen_cnn = tf.keras.preprocessing.image.ImageDataGenerator(validation_split = 0.2)

    train_cnn = datagen_cnn.flow_from_directory(data_dir,
                                          class_mode = "categorical",
                                          target_size = target_size,
                                          color_mode="rgb",
                                          batch_size = batch_size, 
                                          shuffle = shuffle,
                                          subset='training',
                                          seed = seed)

    validation_cnn = datagen_cnn.flow_from_directory(data_dir,
                                          class_mode = "categorical",
                                          target_size = target_size,
                                          color_mode="rgb",
                                          batch_size = batch_size, 
                                          shuffle = shuffle,
                                          subset='validation',
                                          seed = seed)

    datagen_sn = ImageDataGenerator(
                                          rescale=1./255,
                                          shear_range=0.2,
                                          zoom_range=0.2,
                                          horizontal_flip=True,
                                          validation_split = 0.2
                                          )

    test_sn = ImageDataGenerator(rescale=1./255)

    train_sn = datagen_sn.flow_from_directory(
                                          data_dir,
                                          class_mode='categorical',
                                          target_size = target_size,
                                          color_mode="rgb",
                                          batch_size = batch_size,
                                          shuffle = shuffle,
                                          subset='training',
                                          seed = seed)

    validation_sn = datagen_sn.flow_from_directory(
                                          data_dir,
                                          class_mode='categorical',
                                          target_size = target_size,
                                          color_mode="rgb",
                                          batch_size = batch_size,
                                          shuffle = shuffle,
                                          subset='validation',
                                          seed = seed)

    return train_cnn, validation_cnn, train_sn, validation_sn 

## Modelos 

#### Criando bases convlucioinais

Pode-se ver que a saída de cada camada Conv2D e MaxPooling2D é um tensor 3D de forma (altura, largura, canais). As dimensões de largura e altura tendem a diminuir à medida que você se aprofunda na rede. O número de canais de saída para cada camada Conv2D é controlado pelo primeiro argumento (por exemplo, 32 ou 64). Normalmente, à medida que a largura e a altura diminuem, você pode (computacionalmente) adicionar mais canais de saída em cada camada Conv2D. 

#### Adicionar camadas densas no topo

Para completar o modelo, o último tensor de saída da base convolucional (de formato (4, 4, 64)) em uma ou mais camadas Densas para realizar a classificação. Camadas densas recebem vetores como entrada (que são 1D), enquanto a saída atual é um tensor 3D. Primeiro, você nivelará (ou desenrolará) a saída 3D em 1D e, em seguida, adicionará uma ou mais camadas Densas na parte superior. O conjunto tem 3 classes de saída, então você usa uma camada Densa final com 3 saídas.

### CNN

In [ ]:
def Model_CNN():
  model = models.Sequential()

  # Block 1
  model.add(layers.Conv2D(32, (3, 3), activation='relu', padding='same', name='block1_conv1', input_shape=(768, 768, 3)))
  model.add(layers.Conv2D(32, (1, 1), activation='relu',name='block1_conv2'))
  model.add(layers.MaxPooling2D((2, 2),strides=(2, 2), name='block1_pool'))

  # Block 2
  model.add(layers.Conv2D(64, (3, 3), activation='relu', name='block2_conv1'))
  model.add(layers.Conv2D(64, (1, 1), activation='relu',name='block2_conv2'))
  model.add(layers.MaxPooling2D((2, 2),strides=(2, 2), name='block2_pool'))

  # Block 3
  model.add(layers.Conv2D(128, (3, 3), activation='relu',name='block3_conv1'))
  model.add(layers.Conv2D(128, (1, 1), activation='relu',name='block3_conv2'))
  model.add(layers.MaxPooling2D((2, 2),strides=(2, 2), name='block3_pool'))

  # Block 4
  model.add(layers.Conv2D(256, (3, 3), activation='relu',name='block4_conv1'))
  model.add(layers.Conv2D(256, (1, 1), activation='relu',name='block4_conv2'))
  model.add(layers.MaxPooling2D((2, 2),strides=(2, 2), name='block4_pool'))

  # Block 5
  model.add(layers.Conv2D(512, (3, 3), activation='relu',name='block5_conv1'))
  model.add(layers.Conv2D(512, (1, 1), activation='relu',name='block5_conv2'))
  model.add(layers.MaxPooling2D((2, 2),strides=(2, 2), name='block5_pool'))

  # Flatten
  model.add(layers.Flatten())
  model.add(layers.Dense(512, activation='relu'))
  model.add(layers.Dense(256, activation='relu'))
  model.add(layers.Dense(128, activation='relu'))
  model.add(layers.Dense(3, activation='softmax'))
  
  return model

Como visto as (768, 768, 64) saídas foram achatadas em vetores de forma (387) antes de passar por duas camadas densas.

### VGG

In [ ]:
def Model_VGG():
  model = VGG16(include_top=False, input_shape=(768, 768, 3))

  for layer in model.layers:
   layer.trainable = False
  
  flat1 = Flatten()(model.layers[-1].output)
  class1 = Dense(1024, activation='relu')(flat1)
  class2 = Dense(512, activation='relu')(class1)
  class3 = Dense(256, activation='relu')(class2)
  class4 = Dense(128, activation='relu')(class3)
  output = Dense(3, activation='softmax')(class4)

  model = Model(inputs=model.inputs, outputs=output)

  return model

### SqueezeNet

In [ ]:
sq1x1 = "squeeze1x1"
exp1x1 = "expand1x1"
exp3x3 = "expand3x3"
relu = "relu_"

WEIGHTS_PATH = "https://github.com/rcmalli/keras-squeezenet/releases/download/v1.0/squeezenet_weights_tf_dim_ordering_tf_kernels.h5"
WEIGHTS_PATH_NO_TOP = "https://github.com/rcmalli/keras-squeezenet/releases/download/v1.0/squeezenet_weights_tf_dim_ordering_tf_kernels_notop.h5"

def fire_module(x, fire_id, squeeze=16, expand=64):
    s_id = 'fire' + str(fire_id) + '/'

    if K.image_data_format() == 'channels_first':
        channel_axis = 1
    else:
        channel_axis = 3
    
    x = Convolution2D(squeeze, (1, 1), padding='valid', name=s_id + sq1x1)(x)
    x = Activation('relu', name=s_id + relu + sq1x1)(x)

    left = Convolution2D(expand, (1, 1), padding='valid', name=s_id + exp1x1)(x)
    left = Activation('relu', name=s_id + relu + exp1x1)(left)

    right = Convolution2D(expand, (3, 3), padding='same', name=s_id + exp3x3)(x)
    right = Activation('relu', name=s_id + relu + exp3x3)(right)

    x = concatenate([left, right], axis=channel_axis, name=s_id + 'concat')
    return x


def Model_SqzNet(include_top=True, weights='imagenet',
               input_tensor=None, input_shape=None,
               pooling=None,
               use_bn_on_input = False,
               classes=1000):
    """Instantiates the SqueezeNet architecture.
    """
        
    if weights not in {'imagenet', None}:
        raise ValueError('The `weights` argument should be either '
                         '`None` (random initialization) or `imagenet` '
                         '(pre-training on ImageNet).')

    if weights == 'imagenet' and classes != 1000:
        raise ValueError('If using `weights` as imagenet with `include_top`'
                         ' as true, `classes` should be 1000')


    input_shape = _obtain_input_shape(input_shape,
                                      default_size=227,
                                      min_size=48,
                                      data_format=K.image_data_format(),
                                      require_flatten=include_top)

    if input_tensor is None:
        raw_img_input = Input(shape=input_shape)
    else:
        if not K.is_keras_tensor(input_tensor):
            img_input = Input(tensor=input_tensor, shape=input_shape)
        else:
            img_input = input_tensor
    if use_bn_on_input:
        img_input = BatchNormalization()(raw_img_input)
    else:
        img_input = raw_img_input

    x = Convolution2D(64, (3, 3), strides=(2, 2), padding='valid', name='conv1')(img_input)
    x = Activation('relu', name='relu_conv1')(x)
    x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), name='pool1')(x)

    x = fire_module(x, fire_id=2, squeeze=16, expand=64)
    x = fire_module(x, fire_id=3, squeeze=16, expand=64)
    x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), name='pool3')(x)

    x = fire_module(x, fire_id=4, squeeze=32, expand=128)
    x = fire_module(x, fire_id=5, squeeze=32, expand=128)
    x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), name='pool5')(x)

    x = fire_module(x, fire_id=6, squeeze=48, expand=192)
    x = fire_module(x, fire_id=7, squeeze=48, expand=192)
    x = fire_module(x, fire_id=8, squeeze=64, expand=256)
    x = fire_module(x, fire_id=9, squeeze=64, expand=256)
    
    if include_top:
    
        x = Dropout(0.5, name='drop9')(x)

        x = Convolution2D(classes, (1, 1), padding='valid', name='conv10')(x)
        x = Activation('relu', name='relu_conv10')(x)
        x = GlobalAveragePooling2D()(x)
        x = Activation('softmax', name='loss')(x)
    else:
        if pooling == 'avg':
            x = GlobalAveragePooling2D()(x)
        elif pooling=='max':
            x = GlobalMaxPooling2D()(x)
        elif pooling==None:
            pass
        else:
            raise ValueError("Unknown argument for 'pooling'=" + pooling)

    if input_tensor is not None:
        inputs = get_source_inputs(input_tensor)
    else:
        inputs = img_input

    model = Model(inputs, x, name='squeezenet')

    if weights == 'imagenet':
        if include_top:
            weights_path = get_file('squeezenet_weights_tf_dim_ordering_tf_kernels.h5',
                                    WEIGHTS_PATH,
                                    cache_subdir='models')
        else:
            weights_path = get_file('squeezenet_weights_tf_dim_ordering_tf_kernels_notop.h5',
                                    WEIGHTS_PATH_NO_TOP,
                                    cache_subdir='models')
            
        model.load_weights(weights_path)
        if K.backend() == 'theano':
            layer_utils.convert_all_kernels_in_model(model)

        if K.image_data_format() == 'channels_first':

            if K.backend() == 'tensorflow':
                print('You are using the TensorFlow backend, yet you '
                              'are using the Theano '
                              'image data format convention '
                              '(`image_data_format="channels_first"`). '
                              'For best performance, set '
                              '`image_data_format="channels_last"` in '
                              'your Keras config '
                              'at ~/.keras/keras.json.')
    return model

## Compilando o Modelo

In [ ]:
train_cnn, validation_cnn, train_sn, validation_sn = PreProcess(data_dir,target_size,batch_size,shuffle,seed)

#### Função para Compilar a CNN

In [ ]:
def Compile_Model_CNN(loss, optimizer_cnn, metrics):
  model_cnn = Model_CNN()
  model_cnn.compile(loss=loss, optimizer=optimizer_cnn, metrics=[metrics])
  return model_cnn

def Compile_Model_VGG(loss, optimizer_cnn, metrics):
  model_vgg = Model_VGG()
  model_vgg.compile(loss=loss, optimizer=optimizer_cnn, metrics=[metrics])
  return model_vgg

def Compile_Model_SqzNet(input_shape, nb_class, loss, optimizer_sn, metrics):
  model_sn = Model_SqzNet(input_shape=input_shape, weights = None, classes = nb_class)
  sgd = SGD(lr=0.001, decay=0.0002, momentum=0.9, nesterov=True)
  model_sn.compile(loss=loss, optimizer=optimizer_sn, metrics=[metrics])
  return model_sn

### Compilando os modelos

In [ ]:
model_cnn = Compile_Model_CNN(loss, optimizer_cnn, metrics)
model_vgg = Compile_Model_VGG(loss, optimizer_cnn, metrics)
model_sn = Compile_Model_SqzNet(input_shape, nb_class, loss, optimizer_sn, metrics)

## Treinando o Modelo

### Funções de Treinamento

In [ ]:
def Train_Model_CNN(train_cnn, nb_epoch, validation_cnn):
     model_cnn.fit(train_cnn, epochs = nb_epoch,  validation_data=validation_cnn)
     return model_cnn

def Train_Model_VGG(train_cnn, nb_epoch, validation_cnn):
     model_vgg.fit(train_cnn, epochs = nb_epoch,  validation_data=validation_cnn)
     return model_vgg

def Train_Model_SqzNet(train_sn, nb_epoch, validation_sn):
     model_sn.fit(train_sn, epochs = nb_epoch,  validation_data=validation_sn)
     return model_sn

### Treinando a CNN

In [ ]:
history_cnn = Train_Model_CNN(train_cnn, nb_epoch, validation_cnn)

### Treinando a VGG

In [ ]:
history_vgg = Train_Model_VGG(train_cnn, nb_epoch, validation_cnn)

### Treinando a SqueezeNet

In [ ]:
history_sn = Train_Model_SqzNet(train_sn, nb_epoch, validation_sn)

## Avaliando o Modelo

### Avaliando a CNN

In [ ]:
def Evaluate_Model_CNN():
    predicted_cnn = model_cnn.predict(validation_cnn)

    print("F1 Score--------------->")
    print(f1_score(validation_cnn.classes, np.argmax(predicted_cnn, axis = 1), average = 'micro'))
    print(f1_score(validation_cnn.classes, np.argmax(predicted_cnn, axis = 1), average = 'macro'))
    print(f1_score(validation_cnn.classes, np.argmax(predicted_cnn, axis = 1), average = 'weighted'))

    print("Recall ------------>")
    print(recall_score(validation_cnn.classes, np.argmax(predicted_cnn, axis = 1), average = 'micro'))
    print(recall_score(validation_cnn.classes, np.argmax(predicted_cnn, axis = 1), average = 'macro'))
    print(recall_score(validation_cnn.classes, np.argmax(predicted_cnn, axis = 1), average = 'weighted'))
    
    print("Precision ------------>")
    print(precision_score(validation_cnn.classes, np.argmax(predicted_cnn, axis = 1), average = 'micro'))
    print(precision_score(validation_cnn.classes, np.argmax(predicted_cnn, axis = 1), average = 'macro'))
    print(precision_score(validation_cnn.classes, np.argmax(predicted_cnn, axis = 1), average = 'weighted'))

### Avaliando a VGG 

In [ ]:
def Evaluate_Model_VGG():
    predicted_cnn = model_cnn.predict(validation_cnn)

    print("F1 Score--------------->")
    print(f1_score(validation_cnn.classes, np.argmax(predicted_cnn, axis = 1), average = 'micro'))
    print(f1_score(validation_cnn.classes, np.argmax(predicted_cnn, axis = 1), average = 'macro'))
    print(f1_score(validation_cnn.classes, np.argmax(predicted_cnn, axis = 1), average = 'weighted'))

    print("Recall ------------>")
    print(recall_score(validation_cnn.classes, np.argmax(predicted_cnn, axis = 1), average = 'micro'))
    print(recall_score(validation_cnn.classes, np.argmax(predicted_cnn, axis = 1), average = 'macro'))
    print(recall_score(validation_cnn.classes, np.argmax(predicted_cnn, axis = 1), average = 'weighted'))
    
    print("Precision ------------>")
    print(precision_score(validation_cnn.classes, np.argmax(predicted_cnn, axis = 1), average = 'micro'))
    print(precision_score(validation_cnn.classes, np.argmax(predicted_cnn, axis = 1), average = 'macro'))
    print(precision_score(validation_cnn.classes, np.argmax(predicted_cnn, axis = 1), average = 'weighted'))

### Avaliando a SqueezeNet

In [ ]:
def Evaluate_Model_SqzNet():
    predicted_sn = model_sn.predict(validation_sn)

    print("F1 Score--------------->")
    print(f1_score(validation_sn.classes, np.argmax(predicted_sn, axis = 1), average = 'micro'))
    print(f1_score(validation_sn.classes, np.argmax(predicted_sn, axis = 1), average = 'macro'))
    print(f1_score(validation_sn.classes, np.argmax(predicted_sn, axis = 1), average = 'weighted'))

    print("Recall ------------>")
    print(recall_score(validation_sn.classes, np.argmax(predicted_sn, axis = 1), average = 'micro'))
    print(recall_score(validation_sn.classes, np.argmax(predicted_sn, axis = 1), average = 'macro'))
    print(recall_score(validation_sn.classes, np.argmax(predicted_sn, axis = 1), average = 'weighted'))
    
    print("Precision ------------>")
    print(precision_score(validation_sn.classes, np.argmax(predicted_sn, axis = 1), average = 'micro'))
    print(precision_score(validation_sn.classes, np.argmax(predicted_sn, axis = 1), average = 'macro'))
    print(precision_score(validation_sn.classes, np.argmax(predicted_sn, axis = 1), average = 'weighted'))

## Salvando o Modelo

### Salvando a CNN

In [ ]:
model_json = model_cnn.to_json()
model_name = "Modelo_CNN"
with open(model_name+".json", "w") as json_file:
    json_file.write(model_json)
    
model.save_weights(model_name+".h5")

### Salvando a VGG

In [ ]:
model_json = model_vgg.to_json()
model_name = "Modelo_VGG"
with open(model_name+".json", "w") as json_file:
    json_file.write(model_json)
    
model.save_weights(model_name+".h5")

### Salvando a SqueezeNet

In [ ]:
model_json = model_sn.to_json()
model_name = "Modelo_SqueezeNet"
with open(model_name+".json", "w") as json_file:
    json_file.write(model_json)
    
model.save_weights(model_name+".h5")

## Execução 

### Execução da CNN

In [ ]:
def Execute_Model_CNN():
  
  PreProcess(data_dir,target_size,batch_size,shuffle,seed)
  Compile_Model_CNN(loss, optimizer_cnn, metrics)
  Train_Model_CNN(train_cnn, nb_epoch, validation_cnn)
  Evaluate_Model_CNN()

### Execução da VGG

In [ ]:
def Execute_Model_VGG():
 
  PreProcess(data_dir,target_size,batch_size,shuffle,seed)
  Compile_Model_VGG(loss, optimizer_cnn, metrics)
  Train_Model_VGG(train_cnn, nb_epoch, validation_cnn)
  Evaluate_Model_VGG()

### Execução da SqueezeNet

In [ ]:
def Execute_Model_SqzNet():
  PreProcess(data_dir,target_size,batch_size,shuffle,seed)
  Compile_Model_SqzNet(input_shape, nb_class, loss, optimizer_sn, metrics)
  Train_Model_SqzNet(train_cnn, nb_epoch, validation_cnn)
  Evaluate_Model_SqzNet()